In [38]:
import pandas as pd
import numpy as np

In [198]:
class DataFrameProcessor:
    def __init__(self, changing_cats_path, categories_for_extraction_path, output_path):
        self.changing_cats_path = changing_cats_path
        self.categories_for_extraction_path = categories_for_extraction_path
        self.output_path = output_path
        self.changing_cats = None
        self.categories_for_extraction = None

    def load_data(self):
        """Загрузка данных из CSV файлов."""
        try:
            self.changing_cats = pd.read_csv(self.changing_cats_path)
            self.categories_for_extraction = pd.read_csv(self.categories_for_extraction_path)
            print("Данные загружены.")
        except Exception as e:
            print(f"Ошибка при загрузке данных: {e}")

    def clean_data(self):
        """Очистка данных."""
        try:
            # Удаление ненужных столбцов, если они существуют
            cols_to_drop = ["Unnamed: 3", "Unnamed: 4", "Unnamed: 5"]
            self.changing_cats.drop(columns=[col for col in cols_to_drop if col in self.changing_cats.columns], inplace=True)

            # Удаление строк с ненужными категориями
            unwanted_categories = ["Юмор", "Разное"]
            self.changing_cats = self.changing_cats[~self.changing_cats['Ours'].isin(unwanted_categories)]
            self.categories_for_extraction = self.categories_for_extraction[~self.categories_for_extraction['category'].isin(unwanted_categories)]

            print("Очистка данных завершена.")
        except KeyError as e:
            print(f"Ошибка: Не удается найти столбец: {e}")

    def update_tags(self):
        """Обновление тегов на основе категории."""
        try:
            # Создание словарей для замены категорий и тегов
            category_to_dict = self.changing_cats.set_index('Ours')['Rutube lvl0'].to_dict()
            tag_to_dict = self.changing_cats.set_index('Ours')['Tag'].to_dict()

            # Обработка категории "Охота и рыбалка"
            mask = self.categories_for_extraction['category'] == "Охота и рыбалка"
            
            # Только строки с категорией "Охота и рыбалка" изменяем на "Спорт" и добавляем тег "Рыбалка"
            self.categories_for_extraction.loc[mask, 'category'] = "Спорт"
            self.categories_for_extraction.loc[mask, 'tag'] = "Рыбалка"

            # Использование map() для замены категорий и тегов на основе словарей
            self.categories_for_extraction['category'] = self.categories_for_extraction['category'].map(
                category_to_dict).fillna(self.categories_for_extraction['category'])
            self.categories_for_extraction['tag'] = self.categories_for_extraction['category'].map(
                tag_to_dict).fillna(self.categories_for_extraction['tag'])

            print("Обновление тегов завершено.")
        except Exception as e:
            print(f"Ошибка при обновлении тегов: {e}")

    def save_to_csv(self):
        """Сохранение итогового DataFrame в CSV файл."""
        try:
            self.categories_for_extraction.to_csv(self.output_path, index=False, encoding='utf-8')
            print(f"Результаты сохранены в {self.output_path}.")
        except Exception as e:
            print(f"Ошибка при сохранении файла: {е}")

    def process(self):
        """Основной метод для выполнения всех этапов."""
        self.load_data()
        self.clean_data()
        self.update_tags()
        self.save_to_csv()
        self.categories_for_extraction

# Параметры для инициализации класса
changing_cats_path = "/root/EDA/rutube - refactor.csv"
categories_for_extraction_path = "/root/EDA/metadata.csv"
output_path = "metadata_renamed.csv"

# Создание и запуск экземпляра класса
processor = DataFrameProcessor(changing_cats_path, categories_for_extraction_path, output_path)
processor.process()


Данные загружены.
Очистка данных завершена.
Обновление тегов завершено.
Результаты сохранены в metadata_renamed.csv.


In [201]:
categories_for_extraction_path = "/root/EDA/metadata_renamed.csv"
df = pd.read_csv(categories_for_extraction_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7220 entries, 0 to 7219
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   video_id     7220 non-null   object
 1   category     7220 non-null   object
 2   tag          7220 non-null   object
 3   title        7220 non-null   object
 4   description  6817 non-null   object
dtypes: object(5)
memory usage: 282.2+ KB
